In [2]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_dataframe(file_path):
    df = pd.read_parquet(file_path)

    # Calculating the duration in seconds
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td : td.total_seconds() / 60)

    # Taking duration Morethan 1 minute and Lessthan 1 hour
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical and Numerical Variables to use
    categorical = ['PULocationID', 'DOLocationID']

    # One hot encoing Categorical Variables using Dictionary Vectorizer
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
# Loading dataframes
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [5]:
# Creating new columns
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
# List of columns to be used
categorical = ['PU_DO']
numerical = ['trip_distance']

# Initializing DictVectorizer
dv = DictVectorizer()

# Collecting train dicts
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Collecting Val dicts
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
# Collecting train and val y data
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
# Building Model - Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

np.sqrt(mean_squared_error(y_val, y_pred))

7.758715205596356

In [9]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("NYC Taxi Learning")


with mlflow.start_run():

    mlflow.set_tag("developer-name", "Nishan")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./data/green_tripdata_2021-02.parquet")
        
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("RMSE", rmse)

In [33]:
# Saving the model
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [13]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:6.82518                           
[1]	validation-rmse:6.71576                           
[2]	validation-rmse:6.68951                           
[3]	validation-rmse:6.68360                           
[4]	validation-rmse:6.67821                           
[5]	validation-rmse:6.67226                           
[6]	validation-rmse:6.67092                           
[7]	validation-rmse:6.67141                           
[8]	validation-rmse:6.66991                           
[9]	validation-rmse:6.66924                           
[10]	validation-rmse:6.66671                          
[11]	validation-rmse:6.66482                          
[12]	validation-rmse:6.66558                          
[13]	validation-rmse:6.66329                          
[14]	validation-rmse:6.66316                          
[15]	validation-rmse:6.66087                          
[16]	validation-rmse:6.66003                          
[17]	validation-rmse:6.65866                          
[18]	valid

KeyboardInterrupt: 

In [15]:
params = {
"learning_rate": 0.20472169880371677,
"max_depth": 17,
"min_child_weight": 1.2402611720043835,
"objective": "reg:linear",
"reg_alpha": 0.28567896734700793,
"reg_lambda": 0.004264404814393109,
"seed": 42
}

mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000, 
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2024/08/31 20:52:36 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/08/31 20:52:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '689e300e71c84241abeda25aa63110be', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

2024/08/31 20:54:31 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/08/31 20:54:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
